# Avaliação do modelo
este notebook é para ser utilizado para testes generalizados do modelo com melhor performance.

## Pré processamento dos dados

In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import yaml
import json
import fasttext
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import torch
with open("confs.yaml") as f:
    confs = yaml.safe_load(f)

In [2]:
allowed_cols = confs["training_set_schema"] 
df = pd.read_parquet('train.parquet')[allowed_cols] # Subsitua 'train.parquet' pelo caminho correto do seu arquivo

df.head(5)

,user_input,razaosocial,nome_fantasia,uf,identificador_matriz_filial,data_situacao_cadastral,nome_cidade_exterior,data_inicio_atividade,capital_social,porte,cnae_fiscal,descricao_cnae,nome_pais,nome_natureza_juridica,nome_motivo,nome_qualificacao,cnpj
50222,MAGAZINE L,MAGAZINE LUIZA S/A,MAGAZINE LUIZA,SP,filial,0,None,20050317,1.380216e+10,demais,4713004,"Lojas de departamentos ou magazines, exceto lo...",None,Sociedade Anônima Aberta,SEM MOTIVO,Diretor,47960950025388
99431,PNEUS GP,GP PNEUS LTDA,GP PNEUS,PR,filial,20221209,None,20221209,1.156500e+08,demais,4530705,Comércio a varejo de pneumáticos e câmaras-de-ar,None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,46378127006695
240152,SANTA CRUZ DISTRIBUIDORA,DISTRIBUIDORA DE MEDICAMENTOS SANTA CRUZ LTDA,SANTA CRUZ,RS,filial,20051103,None,19970624,6.722510e+08,demais,4644301,Comércio atacadista de medicamentos e drogas d...,None,Sociedade Empresária Limitada,SEM MOTIVO,Administrador,61940292004710
207820,DROGALL,DROGAL FARMACEUTICA LTDA,DROGAL JAGUARIUNA,SP,filial,20220613,None,20220613,5.000000e+06,demais,4771701,"Comércio varejista de produtos farmacêuticos, ...",None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,54375647029451
68386,ESTAPAR BRASIL LTDA,"ALLPARK EMPREENDIMENTOS, PARTICIPACOES E SERVI...",ESTAPAR,ES,filial,20240604,None,20240604,6.456299e+08,demais,5223100,Estacionamento de veículos,None,Sociedade Anônima Aberta,SEM MOTIVO,Presidente,60537263145644


In [3]:
separator_pattern = re.compile(r'[-.,;:()\[\]!?@#$%&*_+=/\\|]+')
punctuation_pattern = re.compile(r'[^\w\s]')  
accent_pattern = re.compile(r'[\u0300-\u036f]') 
space_pattern = re.compile(r'\s+')


def remove_juridical_terms(text, additional_stopwords=None):
    """
    Remove termos jurídicos e palavras irrelevantes de nomes de empresas
    
    Args:
        text: Texto pré-processado
        additional_stopwords: Lista de stopwords adicionais para remover
        
    Returns:
        Texto limpo
    """
    # Termos jurídicos básicos (em minúsculas, sem acentos)
    juridical_terms = {
        'sociedade', 'anonima', 'limitada', 'ltda', 'me', 'epp', 'ep', 
        'sa', 's a', 'comercio', 'industria', 'servicos', 'empresa',
        'falido', 'falencia', 'grupo', 'associacao', 'associados',
        'holding', 'cooperativa', 'nacional', 'internacional', 'brasileira', 'solucoes'
    }
    
    # Adicionar stopwords customizadas se fornecidas
    if additional_stopwords:
        juridical_terms |= set(additional_stopwords)
    
    # Dividir o texto em palavras e filtrar
    words = [word for word in text.split() if word not in juridical_terms]
    
    return ' '.join(words).strip()



def simplify_text(text, keep_accents=False, remove_juridical=True, additional_stopwords=None):
    if not isinstance(text, str) or not text.strip():
        return ""
    
    
    # Converter para minúsculas
    text = text.lower()
    
    # Tratamento especial para S.A. e S/A
    text = re.sub(r'\b(s\.?\s*[aàáâã]\.?|s\s*/\s*a)\b', 'sa', text)
    
    # Normalizar separadores - converter para espaços
    text = separator_pattern.sub(' ', text)
    
    # Tratamento de acentos e caracteres especiais
    if not keep_accents:
        text = unidecode(text)  # Remove acentos e normaliza caracteres
        text = punctuation_pattern.sub('', text)  # Remove pontuação
    else:
        # Remove apenas marcas diacríticas mantendo caracteres base
        text = accent_pattern.sub('', text)
        text = punctuation_pattern.sub('', text)  # Remove pontuação
    
    # Normalizar espaços (remove duplos, mantém únicos)
    text = space_pattern.sub(' ', text).strip()
    
    # Substituir termos jurídicos por extenso
    # Usando regex com \b para garantir palavras completas
    text = re.sub(r'\bsa\b', 'sociedade anonima', text)
    text = re.sub(r'\bltda\b', 'limitada', text)
    
    if remove_juridical:
        text = remove_juridical_terms(text, additional_stopwords)
    
    # Normalizar espaços novamente após substituições
    text = space_pattern.sub(' ', text).strip()
    
    return text
def process_dataframe(df, columns, keep_accents=False):
    """
    Process the DataFrame by applying text simplification to specified columns.
    Processa o dataframe aplicando simplificação de texto nas colunas especificadas.
    
    Args:
        df (pd.DataFrame): dataframe para processar.
        columns (list): Lista de colunas para aplicar simplificação.
    
    Returns:
        pd.DataFrame: dataframe processado simplificado.
    """
    df = df.copy() 
    
    for col in columns:
        new_col = f"{col}_clean"
        df[new_col] = df[col].apply(lambda x: simplify_text(x, keep_accents))
            
    df['combined_input'] = df['user_input_clean'] + " - " + df['uf'].astype(str)
    return df

text_columns = ['razaosocial', 'nome_fantasia', 'user_input']

# Processar dataframe
df_clean = process_dataframe(df, text_columns)

# Mostrar resultado
print(df_clean[['user_input', 'uf', 'combined_input']].head())

                      user_input  uf                 combined_input
50222                 MAGAZINE L  SP                magazine l - SP
99431                   PNEUS GP  PR                  pneus gp - PR
240152  SANTA CRUZ DISTRIBUIDORA  RS  santa cruz distribuidora - RS
207820                   DROGALL  SP                   drogall - SP
68386        ESTAPAR BRASIL LTDA  ES            estapar brasil - ES


In [4]:
def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

words_dict = set()
numbers_found = []  

for col in ['razaosocial_clean', 'nome_fantasia_clean']:
    for text in df_clean[col].dropna():
        words = text.split() 
        for word in words:
            if len(word) >= 4:
                # Identificar palavras que contêm dígitos
                if any(char.isdigit() for char in word):
                    numbers_found.append(word)
                # adicionar a versão normalizada
                words_dict.add(word.lower())

print(f"Total unique words: {len(words_dict)}")

# Mantém apenas palavras que não são números puros
filtered_words = {
    word for word in words_dict 
    if not word.isdigit()  # Remove numeros puros
}

# List the pure numbers that were removed
removed_pure_numbers = [word for word in words_dict if word.isdigit()]
print("\nPure Numbers Removed")
print(removed_pure_numbers)

Total unique words: 8631

Pure Numbers Removed
['2120', '4605', '4587', '4161', '1535', '4683', '3156', '4304', '0978', '0003', '2673', '4655', '4557', '2045', '11595', '4862', '0006', '4448', '4789', '4202', '0438', '4667', '4836', '3916', '9061', '4692', '4567', '13598', '3988', '00313', '4785', '2119', '4608', '4285', '7711', '4445', '2184', '1168', '3490', '4846', '5921', '4336', '06766400609', '13603', '4318', '4142', '0052', '1269', '00229', '1331', '9084', '3387', '3077', '4311', '12453', '4854', '4446', '0550', '5001974', '4828', '4698', '5443989', '3374', '4682', '0251', '11440', '3858', '4519', '4761', '5005088', '4654', '7777', '00189', '2103', '14116', '3030', '4538', '3541', '4451', '4726', '00634', '2795', '3627', '9176', '9714', '1850', '7516', '2046', '13864', '4617', '4827', '4818', '4267', '3129', '3000', '4178', '2882', '11815', '12647', '3554', '2089', '12381', '4769', '00562', '3861', '4510', '1443', '000371', '3843', '3865', '1141', '4494', '2765', '1208', '2719',

In [5]:

import pandas as pd
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
import re
from unidecode import unidecode
import time
import json
import fasttext
from tqdm import tqdm
from collections import defaultdict

# Biencoders

In [6]:
# Configurar dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Carregar dados completos

df_clean['texto_empresa'] = df_clean['razaosocial_clean'] + ' ' + df_clean['nome_fantasia_clean']

# 2. Carregar modelo treinado
model_final = SentenceTransformer('./bert-bi-encoder', device=device)

# 3. Preparar dados de teste (todas as entradas originais)
test_data = []
for _, row in df_clean.iterrows():
    test_data.append({
        'consulta': row['user_input_clean'],
        'empresa': row['texto_empresa']
    })

# 4. Criar catálogo completo de empresas únicas
catalogo_empresas = df_clean['texto_empresa'].unique().tolist()
print(f"Catálogo completo contém {len(catalogo_empresas)} empresas únicas")

# 5. Gerar embeddings do catálogo (uma vez)
print("Gerando embeddings para o catálogo completo...")
embeddings_catalogo = model_final.encode(
    catalogo_empresas,
    batch_size=256,
    show_progress_bar=True,
    convert_to_numpy=True
)

# 6. Mapeamento empresa->índice
empresa_to_index = {empresa: idx for idx, empresa in enumerate(catalogo_empresas)}

# 7. Avaliar modelo
top1_correct = 0
top5_correct = 0

print("\nIniciando avaliação no dataset completo...")
for item in tqdm(test_data):
    consulta = item['consulta']
    empresa_alvo = item['empresa']
    
    # Embedding da consulta
    embedding_consulta = model_final.encode(consulta)
    
    # Calcular similaridades
    similaridades = cosine_similarity(
        [embedding_consulta],
        embeddings_catalogo
    )[0]
    
    # Ordenar resultados
    top_indices = np.argsort(similaridades)[::-1]
    
    # Top 1
    if catalogo_empresas[top_indices[0]] == empresa_alvo:
        top1_correct += 1
        
    # Top 5
    top5_empresas = [catalogo_empresas[i] for i in top_indices[:5]]
    if empresa_alvo in top5_empresas:
        top5_correct += 1

# 8. Calcular métricas
total_exemplos = len(test_data)
top1_acc = top1_correct / total_exemplos
top5_acc = top5_correct / total_exemplos

print("\n" + "="*60)
print(f"RESULTADOS NO DATASET COMPLETO ({total_exemplos} exemplos):")
print(f"Acurácia Top-1: {top1_acc:.4f} ({top1_correct}/{total_exemplos})")
print(f"Acurácia Top-5: {top5_acc:.4f} ({top5_correct}/{total_exemplos})")
print("="*60)

Catálogo completo contém 9905 empresas únicas
Gerando embeddings para o catálogo completo...


Batches:   0%|          | 0/39 [00:00<?, ?it/s]


Iniciando avaliação no dataset completo...


100%|██████████| 255471/255471 [3:03:49<00:00, 23.16it/s]  


RESULTADOS NO DATASET COMPLETO (255471 exemplos):
Acurácia Top-1: 0.6417 (163941/255471)
Acurácia Top-5: 0.8664 (221329/255471)
